In [1]:
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
#import igviz as ig
from jupyter_dash import JupyterDash
from dash.dependencies import Input, Output, State
from dash import html
from dash import dcc
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.colors import to_hex

import numpy as np
import pandas as pd 
from sklearn.preprocessing import MinMaxScaler
import networkx as nx

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
scaler = MinMaxScaler()
dominio_encoder = {'nacional':'Nacional', 'dom_0':'Semi urbano', 'dom_1':'Rural', 'dom_2':'Urbano'}

https://github.com/Ashton-Sidhu/plotly-graph igviz doc

Crear los nodos y eso
https://towardsdatascience.com/plotly-meets-scientific-visualization-8c2074f032cb, https://plotly.com/python/network-graphs/ y https://stackoverflow.com/questions/74607000/python-networkx-plotly-how-to-display-edges-mouse-over-text

subplots: https://stackoverflow.com/a/67291224

### Checar otra forma de graficar que sea estático.

### Crear visualización por dominio
1. Leer los grafos por dominio
2. Crear check por si quiere ver nacional, semiurbano, rural o urbano.
3. Graficar el grafo correspondiente.



# Creo la red

## Cargo los datos

In [4]:
degree_centrality_sorted = pd.read_csv('data/degree_centrality_sorted.csv')
degree_centrality_values = pd.read_csv('data/degree_centrality_values.csv', index_col='Unnamed: 0')
degree_centrality_ranking = pd.read_csv('data/degree_centrality_ranking.csv', index_col='Unnamed: 0')
pagerank_sorted = pd.read_csv('data/pagerank_sorted.csv')
pagerank_values = pd.read_csv('data/pagerank_values.csv', index_col='Unnamed: 0')
pagerank_ranking = pd.read_csv('data/pagerank_ranking.csv', index_col='Unnamed: 0')

## Cargar grafos

In [ ]:
nac = nx.read_gpickle('data/Nacional.gpickle')
regiones = []
for nom in list(dominio_encoder.values())[1:]:
    regiones.append(nx.read_gpickle(f'data/{nom}.gpickle'))

Obtengo las coordenadas 2D del layout semiurbano (porque este tiene la mayoría de las variables necesarias)

## Graficar el grafo

In [12]:
def obtener_puntos_para_graficar_texto_sobre_aristas(a, b, num_puntos):
    longitud_sublinea = (b-a)/(num_puntos+1)
    puntos = []
    
    for i in range(num_puntos):
        puntos.append(a+longitud_sublinea*(i+1))
    
    return puntos

In [13]:
#def crear_layout_para_visualizar_grafo(G1, coords_dict, color, color_method, titulo, region, metrica, num_puntos_edge=5):
def crear_layout_para_visualizar_grafo(G1, color, color_method, titulo, region, metrica,
                                       num_puntos_edge=5):
    ## step2: set the dictionary as the attribute of the node
    #nx.set_node_attributes(G1,coords_dict,'coord')
    nx.set_node_attributes(G1,nx.spring_layout(G1,seed=5),'coord')
    
    # step3: draw the nodes, in plotly we call nodes or edges trace
    node_x = []   # store x coordinates
    node_y = []   # store y coordinates
    node_text = [] # store text when mouse hovers over the node
    for node,node_attr_dict in G1.nodes(data=True):  # recall anatomy 
        x,y = node_attr_dict['coord']
        node_x.append(x)
        node_y.append(y)
        #aqui haria texto de lo que quiero que muestre (node, degree, centralidad, ...)
        if metrica == 'centralidad':
            valor = node_attr_dict['centralidad']
            node_text.append(f'Nodo: {node}\nCentralidad de grado: {valor}') 
        else:
            valor = node_attr_dict['pagerank']
            node_text.append(f'Nodo: {node}\nPagerank: {valor}') 
    node_trace = go.Scatter(name='nodes',
                            x=node_x,
                            y=node_y,
                            mode='markers',
                            hoverinfo='text',
                            text=node_text,
                            marker=dict( #esto determina el color del nodo
                                        color=color_method,
                                        colorscale=color,
                                        size=18,
                                        showscale=True,
                                        colorbar=dict(
                                            title='Centralidad de grado' if metrica=='centralidad' else 'Pagerank',
                                            titleside='right',
                                            thickness=15,
                                            xanchor='left'),
                                       )
                           )
    # step 4: draw the edges and mid point for info
    edge_x, edge_y = [], []
    txt_node_x, txt_node_y, txt_node_txt = [], [], [] #crear nodos transparente para mostrar info edges
    for edge_end1,edge_end2,edge_attr_dict in G1.edges(data=True):
        x0,y0 = G1.nodes[edge_end1]['coord']
        x1,y1 = G1.nodes[edge_end2]['coord']
        edge_x.extend([x0,x1,None])
        edge_y.extend([y0,y1,None])

        peso_redondeado = round(edge_attr_dict['weight'], 4)
        txt_node_x.extend(obtener_puntos_para_graficar_texto_sobre_aristas(x0, x1, num_puntos_edge)) 
        txt_node_y.extend(obtener_puntos_para_graficar_texto_sobre_aristas(y0, y1, num_puntos_edge)) 
        #txt_node_x.extend([(x0 + x1)/2]) 
        #txt_node_y.extend([(y0 + y1)/2]) 
        txt_node_txt.extend([f'PFI: {peso_redondeado}\n{edge_end1}->{edge_end2}']*num_puntos_edge) # hovertext

    edge_trace = go.Scatter(name='lines',
                            x=edge_x,
                            y=edge_y,
                            mode='lines',
                            line=dict(width=0.5, color='#888')
                           ) 
    txt_node_trace = go.Scatter(x = txt_node_x,
                                y = txt_node_y,
                                mode = "markers",
                                showlegend = False,
                                hovertemplate = "%{hovertext}<extra></extra>",
                                hovertext = txt_node_txt,
                                opacity=0,
                                marker=dict(color='#888')
                               )
    
    # step5: draw layout
    fig_layout = go.Layout(showlegend=False,
                           title=titulo,
                           xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                           yaxis=dict(showgrid=False, zeroline=False, showticklabels=False)
                          )

    return node_trace, edge_trace, txt_node_trace, fig_layout

## Hago los datos para las Bump Chart

In [14]:
def obtener_df_bumpChart_plotly(df_values, df_ranking, metrica):
    regiones = ['nacional', 'dom_1', 'dom_0', 'dom_2']
    #traduccion_nombres_regiones = ['Nacional', 'Rural', 'Semiurbano', 'Urbano']
    df_parciales = [] #variable que almacena el df resultante por cada domino
    df_values= df_values.reset_index(inplace=False) # para tener una columna con las variables

    for col in regiones:
        temp = df_values[['index', col]] #obtengo las columnas de variable y valor pfi
        temp.columns = ['variable', metrica] #sustituyo el nombre de las columnas
        temp["rank"] = df_ranking[col].values  #obtengo el ranking de la variable
        temp['region'] = dominio_encoder[col] # creo una columna con nombre de la region 
        df_parciales.append(temp)
    
    return pd.concat(df_parciales)

def obtener_colormap_bumpchart(df_sorted, dom, metrica):
    #color = 'nipy_spectral'
    color = 'viridis' if metrica=='centralidad' else 'RdPu'
    return dict(zip(list(df_sorted[dom]),
           #list(sns.color_palette(palette='nipy_spectral',n_colors=50).as_hex())
           #cm.get_cmap('nipy_spectral', 50)(range(50)),
           [to_hex(rgba_color, keep_alpha=False) for rgba_color in cm.get_cmap(color, 50)(range(50,0,-1))]
           ))

In [15]:
df_bumpChart_centralidad = obtener_df_bumpChart_plotly(degree_centrality_values, 
                                                       degree_centrality_ranking,
                                                       'centralidad')
df_bumpChart_pagerank = obtener_df_bumpChart_plotly(pagerank_values,
                                                    pagerank_ranking,
                                                    'pagerank')

In [16]:
cmaps_pagerank = {}
cmaps_centralidad = {}
for dom in pagerank_sorted.columns:
    cmaps_pagerank[dominio_encoder[dom]]   = obtener_colormap_bumpchart(pagerank_sorted, dom, 'centralidad')
    cmaps_centralidad[dominio_encoder[dom]] = obtener_colormap_bumpchart(degree_centrality_sorted, dom, 'pagerank')

#hacer 1 solo df con ambos ranking

## Hago el tablero de Dash

In [17]:
tablero = JupyterDash(__name__)

# -----------------------
# Componentes del tablero
# -----------------------
grafo = dcc.Graph(id='grafo')
bump_chart = dcc.Graph(id='bump_chart')
menuGrafo = dcc.Dropdown(id = 'menuGrafo',
                     options = [{'label':'Centralidad de grado', 'value':'centralidad'},
                                {'label':'Pagerank', 'value':'pagerank'}],
                     multi=False,
                     value='centralidad'
                          )
menuBump = dcc.Dropdown(id = 'menuBump',
                     options = [{'label':'Centralidad de grado', 'value':'centralidad'},
                                {'label':'Pagerank', 'value':'pagerank'}],
                     multi=False,
                     value='centralidad'
                          )
menuRegionesGrafo = dcc.RadioItems(id='menuRegionesGrafo',
                              options=[
                                  {'label': 'Nacional', 'value': 'nacional'},
                                  {'label': 'Rural', 'value': 'dom_1'},
                                  {'label': 'Semiurbano', 'value': 'dom_0'},
                                  {'label': 'Urbano', 'value': 'dom_2'},
                              ],
                              value='nacional',
                              inline=True
            )


menuRegionesBump = dcc.RadioItems(id='menuRegionesBump',
                              options=[
                                  {'label': 'Nacional', 'value': 'Nacional'},
                                  {'label': 'Rural', 'value': 'Rural'},
                                  {'label': 'Semiurbano', 'value': 'Semi urbano'},
                                  {'label': 'Urbano', 'value': 'Urbano'},
                              ],
                              value='Nacional',
                              inline=True
            )

# -----------------------
# Diseño del tablero
# -----------------------
tablero.layout = html.Div([
    html.Div([html.H1('Violencia obstétrica durante el parto en instituciones de salud públicas de México'),
              menuGrafo, 
              menuRegionesGrafo, 
              html.Button('Generar', id='botonGrafo', n_clicks=0), 
              grafo]),
    
    html.Div([html.H3('Gráfica Bump'), 
              menuBump, 
              menuRegionesBump,
              html.Button('Generar', id='botonBump', n_clicks=0), 
              bump_chart]),
])

# -----------------------
# Callbacks
# -----------------------
@tablero.callback(Output('grafo', 'figure'),
                  Input('botonGrafo', 'n_clicks'),
                  State('menuGrafo', 'value'),
                  State('menuRegionesGrafo', 'value'))
def graficarFuncion(n_clicks, metrica, region):
    if region[-1] in ['0', '1', '2']:
        G = regiones[int(region[-1])]
    else:
        G = nac
        
    if metrica=='centralidad':
        titulo = "Representación visual de los grafos con centralidad de grado"
        color = 'RdPu'
        color_method = [degree_centrality_sorted[region][degree_centrality_sorted[region]==col].index[0] 
                        for col in G.nodes]
    else:
        titulo = "Representación visual de los grafos con pagerank"
        color = 'Viridis'
        color_method = [pagerank_sorted[region][pagerank_sorted[region]==col].index[0] for col in G.nodes]
    
    node_trace, edge_trace, txt_node_trace, fig_layout = crear_layout_para_visualizar_grafo(G, 
                                                                                            #coords_dict, 
                                                                                            color,
                                                                                            color_method, 
                                                                                            titulo,
                                                                                            'dom_0', 
                                                                                            metrica)
    return go.Figure(data=[node_trace, edge_trace, txt_node_trace],
                     layout=fig_layout)

@tablero.callback(Output('bump_chart', 'figure'),
                  Input('botonBump', 'n_clicks'),
                  State('menuBump', 'value'),
                  State('menuRegionesBump', 'value'))
def graficarBump(n_clicks, metrica, region):
    df_usar = df_bumpChart_centralidad if metrica=='centralidad' else df_bumpChart_pagerank
    cmap_usar = cmaps_centralidad if metrica=='centralidad' else cmaps_pagerank
    
    #creo los nombres de los subplots
    doms = ['Nacional', 'Semi urbano', 'Rural', 'Urbano']
    doms.pop(doms.index(region))
    subplot_titles = [f'{region} - {doms[i]}' for i in range(len(doms))]
    
    fig = make_subplots(rows=1, cols=3, 
                        subplot_titles=subplot_titles,
                        shared_yaxes=True,
                        y_title='Rank',
                        horizontal_spacing=0.02)
    for i,dom in enumerate(doms):
        figura = px.line(pd.concat([df_usar[(df_usar.region==region)],
                                    df_usar[(df_usar.region==dom)]]),
                          x = 'region',
                          y='rank',
                          color = 'variable',
                          color_discrete_map= cmap_usar[region], 
                          markers=True,
                          hover_data = ['rank', metrica]
                         )
        figura.update_yaxes(visible=False, showticklabels=False)
    
        for traces in [figura["data"][trace] for trace in range(len(figura["data"]))]: 
            fig.append_trace(traces, row=1, col=i+1)
    
    fig.update_yaxes(autorange='reversed')
    fig.update_xaxes(title='', visible=True, showticklabels=True)
    fig.update_layout(showlegend=False)
    
    return fig

if __name__=='__main__':
    tablero.run_server(port=8050, debug=True, use_reloader=False)


Dash app running on http://127.0.0.1:8050/
